In [1]:
from pyspark.sql import SparkSession
from malaya.tokenizer import SentenceTokenizer
from pyspark.sql.functions import array, col, concat_ws
from pyspark.sql import Row
from itertools import chain
import re

class DE_Transformation:
    def __init__(self):
        self.spark = None


# Example list of strings

    # Function to remove special characters
    def remove_special_characters(text):
        return re.sub(r'[^A-Za-z0-9\s]', '', text)


    
    def readRawData(self):
        self.spark = SparkSession.builder.appName("DataTransformationWithMalaya").getOrCreate()
    
        df = self.spark.read.parquet("DE-prj/RawData")
        #drop duplicated
        df = df.drop_duplicates(['url']).select(['title','articleBrief','contents'])
        
        #Merge DF
        df = df.withColumn("Flattened_Contents", concat_ws(" ", col("contents")))
        df = df.select(['title','articleBrief','Flattened_Contents'])
        merged_df = df.withColumn("Merged", array(*[col(c) for c in df.columns])).select(['Merged'])
        lists = merged_df.collect()
        
        # Flatten the 'contents' column
        flattened_list = list(chain.from_iterable(row.Merged for row in lists))

        # Split into sentences
        s_tokenizer = SentenceTokenizer()
        rddInput = self.spark.sparkContext.parallelize(flattened_list)

        # get output in list
        rdd_result = rddInput.map(s_tokenizer.tokenize)
        rdd_result = rdd_result.coalesce(1)
        result = rdd_result.collect()

        return result




    
    
    # def sentenceSegmentationWithSpark(self, df):
    #     try:
    #         self.spark = SparkSession.builder.appName("DataTransformationWithMalaya").getOrCreate()
    #         lists = df.collect()
    #         # Flatten the 'contents' column
    #         flattened_list = list(chain.from_iterable(row.Merged for row in lists))
    #         print(len(flattened_list))
    #         s_tokenizer = SentenceTokenizer()
    #         rddInput = self.spark.sparkContext.parallelize(flattened_list)
    #         # removedSpecialChar = rddInput.map(DE_Transformation.remove_special_characters)
    #         # df = removedSpecialChar.collect()
    #         result = []
    #         for strr in flattened_list:
    #             oneResult = s_tokenizer.tokenize(strr)
    #             if len(oneResult) > 0:
    #                 result.append(oneResult)
    #         # rddInput = self.spark.sparkContext.parallelize(df)
    #         result = rddInput.map(DE_Transformation().sentenceSegmentationProcess).take(30)
    #     finally:
    #         self.spark.stop()

    #     return result

    def sentenceSegmentationProcess(self,strList):
        s_tokenizer = SentenceTokenizer()
        # result = []
        # for ele in strList:
        #     oneResult = s_tokenizer.tokenize(strList)
        #     if len(oneResultreturn oneResult0:
        #         result.append(oneResult)

        # return result
        oneResult = s_tokenizer.tokenize(strList)
        if len(oneResult) > 0:
            return oneResult
        else: 
            return ["",'']

    def sentenceSegmentation(self, df):
        lists = df.collect()
        # Flatten the 'contents' column
        flattened_list = list(chain.from_iterable(row.Merged for row in lists))
        s_tokenizer = SentenceTokenizer()
        result = []
        for str in flattened_list:
            result.append(s_tokenizer.tokenize(str))


        return result 
        
results = DE_Transformation().readRawData()


#print(result.collect())

/home/student/de-prj/de-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
24/12/12 22:10:43 WARN Utils: Your hostname, Liang. resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/12 22:10:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/12/12 22:10:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java

468


[Stage 4:>                                                          (0 + 1) / 1]/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
                                                                                

In [3]:
for ele in results:
    print(ele)

['SPRM diminta utamakan siasatan dakwaan skandal rasuah ADUN Sabah.']
['Gambar hiasan.', '- Foto fail NSTP.']
['KOTA KINABALU: DAP Sabah mahu Suruhanjaya Pencegahan Rasuah Malaysia (SPRM) mengutamakan siasatan dakwaan skandal rasuah membabitkan beberapa Ahli Dewan Undangan Negeri (ADUN) di negeri itu.', 'Setiausahanya, Vivian Wong berkata pihaknya memantau teliti perkembangan kes itu sejak rakaman video mengenainya didedahkan sebuah portal berita sejak sebulan lalu.', 'Katanya, DAP Sabah memandang serius dakwaan yang ditularkan melalui klip video terbabit dan menyokong penuh siasatan SPRM.', '"DAP Sabah memantau dengan teliti perkembangan berkaitan perkara ini dan melihat pendedahan rakaman video baru-baru ini dengan penuh serius.', '"DAP Sabah menyokong penuh SPRM dalam menjalankan siasatan yang teliti dan saksama.', '"Kami menggesa SPRM untuk mengutamakan kes ini dan memastikan ia dikendalikan dengan ketelusan dan integriti," katanya dalam satu kenyataan, di sini.', 'Dakwaan skandal 

In [4]:
spark = SparkSession\
                .builder\
                .appName("DE_malay_news")\
                .getOrCreate()

spark.stop()